In [54]:
import logging
import time
import random
from datetime import datetime

name = 'CAFI'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('batimento')
logger.setLevel(logging.DEBUG)

# fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_batimento.log', mode='w')
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# fHandler.setFormatter(formatter)
# logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {name}')
# logger.debug('debug message')
# logger.warning('warn message')

In [55]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {name}')
levantamento = pd.read_excel('batimento/lev_fisico/2024_levantamento_final.xlsx', sheet_name=1)
levantamento = levantamento.rename(columns={"Código":"codigo"})
# display(levantamento.info())

levantamento = levantamento[["codigo", "ED", "Nome", "Quantidade", "CADMAT", "valor unitário"]]
levantamento = levantamento.drop_duplicates(keep="last")
duplicada = levantamento.duplicated(subset=['codigo'])
# display(len(duplicada))
display(levantamento.info())

suap = pd.read_excel('batimento/suap/2024-01-08_15-39_CAFI_relatorio_almox.xlsx')
display(suap.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403 entries, 0 to 402
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   codigo          403 non-null    object 
 1   ED              403 non-null    int64  
 2   Nome            403 non-null    object 
 3   Quantidade      403 non-null    int64  
 4   CADMAT          396 non-null    float64
 5   valor unitário  0 non-null      float64
dtypes: float64(2), int64(2), object(2)
memory usage: 22.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo     411 non-null    object 
 1   descricao  411 non-null    object 
 2   und        411 non-null    object 
 3   qtd        411 non-null    int64  
 4   vlr_total  411 non-null    float64
 5   vlr_unt    411 non-null    float64
 6   ed         411 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 22.6+ KB


None

In [56]:
batimento = pd.merge(suap, levantamento, on='codigo', how='outer', indicator=True)
display(batimento.info())
duplicada = batimento[batimento.duplicated(subset=['codigo'])]
display(duplicada)
batimento = batimento.drop_duplicates(subset=['codigo'], keep='last')
display(batimento.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 411
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   codigo          412 non-null    object  
 1   descricao       412 non-null    object  
 2   und             412 non-null    object  
 3   qtd             412 non-null    int64   
 4   vlr_total       412 non-null    float64 
 5   vlr_unt         412 non-null    float64 
 6   ed              412 non-null    object  
 7   ED              403 non-null    float64 
 8   Nome            403 non-null    object  
 9   Quantidade      403 non-null    float64 
 10  CADMAT          396 non-null    float64 
 11  valor unitário  0 non-null      float64 
 12  _merge          412 non-null    category
dtypes: category(1), float64(6), int64(1), object(5)
memory usage: 42.4+ KB


None

,codigo,descricao,und,qtd,vlr_total,vlr_unt,ed,ED,Nome,Quantidade,CADMAT,valor unitário,_merge
367,30.26.480,"CABO ELÉTRICO FLEXIVEL 1,5MM VERMELHO TENSÃO 7...",ROLO,3,1.23,0.41,26 - MATERIAL ELETRICO E ELETRONICO,26.0,"CABO ELÉTRICO FLEXIVEL 1,5MM VERMELHO TENSÃO 7...",3.0,1781014.0,NaN,both


<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 411
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   codigo          411 non-null    object  
 1   descricao       411 non-null    object  
 2   und             411 non-null    object  
 3   qtd             411 non-null    int64   
 4   vlr_total       411 non-null    float64 
 5   vlr_unt         411 non-null    float64 
 6   ed              411 non-null    object  
 7   ED              402 non-null    float64 
 8   Nome            402 non-null    object  
 9   Quantidade      402 non-null    float64 
 10  CADMAT          395 non-null    float64 
 11  valor unitário  0 non-null      float64 
 12  _merge          411 non-null    category
dtypes: category(1), float64(6), int64(1), object(5)
memory usage: 42.3+ KB


None

In [57]:
batimento.to_excel(f'batimento/merge/{data_e_hora}_{name}_batimento.xlsx', index=False)

display(batimento['codigo'])

0      30.14.117
1      30.14.121
2      30.14.122
3      30.14.123
4       30.14.13
         ...    
407    30.42.969
408    30.42.970
409    30.42.982
410    30.42.995
411     30.44.67
Name: codigo, Length: 411, dtype: object

In [58]:
ed = batimento['codigo'].str[3:5]
batimento['elemento'] = ed

In [59]:
batimento.to_excel(f'batimento/merge/{data_e_hora}_{name}_batimento.xlsx', index=False)